In [8]:
import pandas as pd
import numpy as np
from sklearn import preprocessing

In [3]:
df = pd.read_csv('Data\Concrete_Data_Yeh.csv')
df.head()

,cement,slag,flyash,water,superplasticizer,coarseaggregate,fineaggregate,age,csMPa
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1030 entries, 0 to 1029
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   cement            1030 non-null   float64
 1   slag              1030 non-null   float64
 2   flyash            1030 non-null   float64
 3   water             1030 non-null   float64
 4   superplasticizer  1030 non-null   float64
 5   coarseaggregate   1030 non-null   float64
 6   fineaggregate     1030 non-null   float64
 7   age               1030 non-null   int64  
 8   csMPa             1030 non-null   float64
dtypes: float64(8), int64(1)
memory usage: 72.5 KB


In [5]:
df.describe()

,cement,slag,flyash,water,superplasticizer,coarseaggregate,fineaggregate,age,csMPa
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [12]:
df_p0 = preprocessing.normalize(df, axis=0)
df_p = pd.DataFrame(df_p0, columns=df.columns)
df_p.head()

,cement,slag,flyash,water,superplasticizer,coarseaggregate,fineaggregate,age,csMPa
0,0.056096,0.000000,0.0,0.027611,0.009046,0.033201,0.027083,0.011197,0.063069
1,0.056096,0.000000,0.0,0.027611,0.009046,0.033680,0.027083,0.011197,0.048798
2,0.034541,0.039097,0.0,0.038860,0.000000,0.029754,0.023798,0.107968,0.031751
3,0.034541,0.039097,0.0,0.038860,0.000000,0.029754,0.023798,0.145956,0.032366
4,0.020631,0.036326,0.0,0.032724,0.000000,0.031235,0.033073,0.143957,0.034929


In [ ]:
df_crom = 

In [ ]:
def strength_funct(df, x1, x2, x3,  x4,  x5,  x6,  x7,  x8):
    csMPa = x1*df['cement'] + x2*df['slag'] + x3*df['flyash'] + x4*df['water'] + x5*df['superplasticizer']
    + x6*df['coarseaggregate'] + x7*df['fineaggregate'] + x8*df['age']

In [ ]:
df['csMPa_est'] = 

In [ ]:
def aic_criterion(df_est):
    return(nrow(df_est)*(log(2*pi)+1+log((sum(df_est['residuals']^2)/nrow(df_est))))
       +((length(df_est.drop(columns=[['residuals', 'csMPa', 'csMPa_est']]).columns)+1)*2)